# Model Tuning

### Load data & imports + Environment check

In [3]:
import os
import sys
from collections import Counter

import numpy as np
import pandas as pd

from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import f1_score, make_scorer

import xgboost as xgb
import joblib

print("Environment check:")
print("Python:", sys.version.split()[0])
print("Pandas:", pd.__version__)
print("NumPy:", np.__version__)
print("XGBoost:", xgb.__version__)


Environment check:
Python: 3.10.18
Pandas: 2.3.3
NumPy: 2.2.6
XGBoost: 3.0.5


### Load Train Split

In [4]:
cwd = os.getcwd()
if os.path.basename(cwd) == "notebooks":
    project_root = os.path.dirname(cwd)
else:
    project_root = cwd

print("Project root:", project_root)

data_dir = os.path.join(project_root, "data", "processed")
train_path = os.path.join(data_dir, "train.csv")

if not os.path.exists(train_path):
    raise FileNotFoundError(f"Missing {train_path}. Run 01_preprocessing.ipynb first.")

train_df = pd.read_csv(train_path)
print("Train shape:", train_df.shape)
train_df.head()


Project root: C:\Users\MayaW\OneDrive - North Dakota University System\UND 2025-2026\CS492 - Senior Project\Final Notebooks
Train shape: (40299, 665)


,role,b_unit,f_unit,dept,team,ITAdmin,O,C,E,A,...,weekendhttp_hackf_mean_url_len,weekendhttp_hackf_mean_url_depth,weekendhttp_hackf_mean_http_c_len,weekendhttp_hackf_mean_http_c_nwords,weekendhttp_hackf_n-pc0,weekendhttp_hackf_n-pc1,weekendhttp_hackf_n-pc2,weekendhttp_hackf_n-pc3,insider,label
0,39,0,4,15,29,0,46,20,32,26,...,0,0,0,0,0,0,0,0,0,0
1,39,0,4,15,4,0,36,41,45,19,...,0,0,0,0,0,0,0,0,0,0
2,39,0,4,15,11,0,11,20,39,20,...,0,0,0,0,0,0,0,0,0,0
3,39,0,1,14,37,0,26,23,16,30,...,0,0,0,0,0,0,0,0,0,0
4,24,0,1,5,28,0,27,35,20,36,...,0,0,0,0,0,0,0,0,0,0


### Build feature matrix 

In [5]:
if "label" not in train_df.columns:
    raise KeyError("Expected 'label' column in train.csv")

y_train = train_df["label"].astype(int)

drop_cols = ["label"]
if "insider" in train_df.columns:
    drop_cols.append("insider")

X_train = train_df.drop(columns=drop_cols)

print("X_train:", X_train.shape)
print("y_train:", y_train.shape)
print("\nLabel distribution:")
print(y_train.value_counts())


X_train: (40299, 663)
y_train: (40299,)

Label distribution:
label
0    40109
1      190
Name: count, dtype: int64


In [6]:
counter = Counter(y_train)
n_neg = counter[0]
n_pos = counter[1]
scale_pos_weight = n_neg / n_pos

print("Class counts:", counter)
print(f"scale_pos_weight: {scale_pos_weight:.2f}")


Class counts: Counter({0: 40109, 1: 190})
scale_pos_weight: 211.10


### Define Base XGBoost Model and Search Space

In [7]:
xgb_base = xgb.XGBClassifier(
    objective="binary:logistic",
    eval_metric="logloss",
    random_state=42,
    n_jobs=-1,
)

spw = scale_pos_weight
param_dist = {
    "learning_rate": [0.03, 0.05, 0.07, 0.1],
    "max_depth": [3, 4, 5, 6],
    "min_child_weight": [1, 3, 5, 7],
    "subsample": [0.7, 0.8, 0.9, 1.0],
    "colsample_bytree": [0.6, 0.7, 0.8, 1.0],
    "n_estimators": [300, 400, 500, 600, 700],
    "gamma": [0, 0.5, 1, 3],
    "reg_lambda": [1, 3, 5, 10],
    "reg_alpha": [0, 0.1, 0.5, 1.0],
    "scale_pos_weight": [
        0.5 * spw,
        0.75 * spw,
        1.0 * spw,
        1.25 * spw,
    ],
}

### Setup Randomized CV Search

In [8]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

f1_scorer = make_scorer(f1_score)

search = RandomizedSearchCV(
    estimator=xgb_base,
    param_distributions=param_dist,
    n_iter=40,              
    scoring=f1_scorer,
    cv=cv,
    n_jobs=-1,
    verbose=1,
    random_state=42,
)


### Run Tuning
This one will take a while too, as it has to evaluate 200 fits. Again, don't be afraid!

**If you don't want to wait, you can hardcode the best params provided in the README**

In [9]:
search.fit(X_train, y_train)

print("Best mean CV F1:", search.best_score_)
print("Best params:")
search.best_params_


Fitting 5 folds for each of 40 candidates, totalling 200 fits
Best mean CV F1: 0.7143576006609965
Best params:


{'subsample': 0.7,
 'scale_pos_weight': 158.325,
 'reg_lambda': 3,
 'reg_alpha': 0,
 'n_estimators': 300,
 'min_child_weight': 1,
 'max_depth': 6,
 'learning_rate': 0.1,
 'gamma': 0,
 'colsample_bytree': 0.8}

### Train tuned model using best parameters

In [11]:
best_params = search.best_params_

#to hard code params
'''best_params = {'subsample': 0.7,
 'scale_pos_weight': 158.325,
 'reg_lambda': 3,
 'reg_alpha': 0,
 'n_estimators': 300,
 'min_child_weight': 1,
 'max_depth': 6,
 'learning_rate': 0.1,
 'gamma': 0,
 'colsample_bytree': 0.8}
 '''
xgb_tuned = xgb.XGBClassifier(
    **best_params,
    objective="binary:logistic",
    eval_metric="logloss",
    random_state=42,
    n_jobs=-1,
)

xgb_tuned.fit(X_train, y_train)


,objective,'binary:logistic'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,0.8
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,'logloss'


### Save tuned model & Params

In [12]:
models_dir = os.path.join(project_root, "models")
os.makedirs(models_dir, exist_ok=True)

model_path = os.path.join(models_dir, "xgb_tuned_model.joblib")
joblib.dump(xgb_tuned, model_path)

print("Saved tuned model to:", model_path)

Saved tuned model to: C:\Users\MayaW\OneDrive - North Dakota University System\UND 2025-2026\CS492 - Senior Project\Final Notebooks\models\xgb_tuned_model.joblib


In [13]:
import json

params_path = os.path.join(models_dir, "xgb_best_params.json")
with open(params_path, "w") as f:
    json.dump(best_params, f, indent=4)

print("Saved best params to:", params_path)
best_params

Saved best params to: C:\Users\MayaW\OneDrive - North Dakota University System\UND 2025-2026\CS492 - Senior Project\Final Notebooks\models\xgb_best_params.json


{'subsample': 0.7,
 'scale_pos_weight': 158.325,
 'reg_lambda': 3,
 'reg_alpha': 0,
 'n_estimators': 300,
 'min_child_weight': 1,
 'max_depth': 6,
 'learning_rate': 0.1,
 'gamma': 0,
 'colsample_bytree': 0.8}

# Conclusion
In this notebook, we tuned the XGBoost classifier using a 20-iteration
`RandomizedSearchCV` over a 5-fold Stratified Cross-Validation split on the
training data.

The search optimized for F1 score, which balances precision and recall on
the highly imbalanced CERT insider-threat dataset. The best parameters were then used to train a final, tuned model on the training set.

The tuned model and its hyperparameters are saved to:

- `models/xgb_tuned_model.joblib`
- `models/xgb_best_params.json`

These will be used in:

- Notebook 04 for feature importance and feature selection
- Notebook 05 for threshold evaluation and calibration